In [4]:
import re
import os
import pathlib
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import gridspec
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
sns.set_theme(style="darkgrid")

In [5]:
# Root of the project
project_dir = pathlib.Path().resolve().parent

# Directory containing experiment results
results_dir = (pathlib.Path(project_dir) / "results").resolve()

# Directory for tidied data (output directory)
data_dir = (pathlib.Path(project_dir) / "data").resolve()

# Directory to store all plots
plot_dir = (pathlib.Path(project_dir) / "plots").resolve()

In [25]:
df = pd.read_csv(data_dir / "fortio_results.csv")
df = df[df["experiment"] == "01_http_max_throughput"]
df = df.reset_index()
df

,index,experiment,mesh,requested_qps,protocol,payload,actual_qps,histogram
0,20,01_http_max_throughput,Cilium,MAX,http,0,18009.233443,"[{'Start': 5.6295e-05, 'End': 0.0001, 'Percent..."
1,21,01_http_max_throughput,Istio,MAX,http,0,4324.863928,"[{'Start': 0.000289687, 'End': 0.0003000000000..."
2,22,01_http_max_throughput,Linkerd,MAX,http,0,9756.589185,"[{'Start': 0.000158514, 'End': 0.0002, 'Percen..."
3,23,01_http_max_throughput,Baseline,MAX,http,0,21652.916972,"[{'Start': 4.6782e-05, 'End': 0.0001, 'Percent..."
4,24,01_http_max_throughput,Traefik,MAX,http,0,565.807761,"[{'Start': 0.000327453, 'End': 0.0004, 'Percen..."


In [26]:
def sample_hist(hist_data):
    multiplier = 1000

    sample_arrays = []
    print(hist_data[1])
    for d in hist_data:
        print(d)
        low = d["Start"] * multiplier
        high = d["Start"] * multiplier
        size = d["Count"]
        sample_arrays.append(np.random.uniform(low=low, high=high, size=size))

    samples = np.concatenate(sample_arrays)

    return samples

In [28]:
df["histogram"].apply(sample_hist)

{
[


TypeError: string indices must be integers